# Assignment 1: Introduction to the Fully Recurrent Network

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.


## Exercise 1: Numerical stability of the binary cross-entropy loss function

We will use the binary cross-entropy loss function to train our RNN, which is defined as 
$$
L_{\text{BCE}}(\hat y, y) = -y \log \hat y - (1-y) \log (1-\hat y),
$$
where $y$ is the label and $\hat y$ is a prediction, which comes from a model (e.g. an RNN) and is usually sigmoid-activated, i.e., we have
$$
\hat y = \sigma(z) = \frac{1}{1+e^{-z}}.
$$
The argument $z$ is called *logit*. For reasons of numerical stability it is better to let the model emit the logit $z$ (instead of the prediction $\hat y$) and incorporate the sigmoid activation into the loss function. Explain why this is the case and how we can gain numerical stability by combining the two functions $L_{\text{BCE}}(\hat y, y)$ and $\sigma(z)$ into one function $L(z, y) = L_{\text{BCE}}(\sigma(z), y)$. 

*Hint: Prove that $\log(1+e^{z}) = \log (1+e^{-|z|}) + \max(0, z)$ and argue why the right-hand side is numerically more stable. Finally, express $L(z,y)$ in terms of that form.*

########## YOUR SOLUTION HERE ##########

1.) Show $\log(1 + e^z) = \log(1 + e^{-|z|}) + \max(0,z)$

if $x <= 0$ this is obvious. 

if $x > 0$
$$ \log(1 + e^{-|z|}) + \max(0,z) = \log(1 + e^{-z}) + z$$
$$ = \log(1 + \frac{1}{e^z}) + z$$
$$ = \log(\frac{e^z + 1}{e^z}) + z$$
$$ = \log(1 + e^z) - z + z = \log(1 + e^z) $$

2.) The right-hand side is numerically more stable because if $z$ gets very large, then $e^z$ will get to large for a float value. You get "inf"

3.) $$ L(z,y) = -y\log\sigma(z) - (1-y)\log(1-\sigma(z))$$
$$  \log \frac{1}{1 + e^{-z}} = -\log(1 + e^{-z}) = -\log(1 + e^{-|z|}) - \max(0,-z) $$
$$  \log (1 - \frac{1}{1 + e^{-z}}) = \log \frac{e^{-z}}{1 + e^{-z}} = -z - \log(1 + e^{-z}) = -z - \log(1 + e^{-|z|}) - \max(0,-z) $$

Therefore we get:
$$ L(z,y) = -y(\log \frac{1}{1 + e^{-|z|}} - \max(0,-z)) - (1-y)(-z + \log \frac{1}{1 + e^{-|z|}} - \max(0,-z)) $$
$$ = -(-z + \log \frac{1}{1 + e^{-|z|}} - \max(0,-z))  - yz $$

$$ = z + \log(1 + e^{-|z|}) + \max(0,-z) - yz$$

## Exercise 2: Derivative of the loss

Calculate the derivative of the binary cross-entropy loss function $L(z, y)$ with respect to the logit $z$.

########## YOUR SOLUTION HERE ##########


In [59]:
import numpy as np
y = 55
z = -5
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def l1(y,z):
    y_h = sigmoid(z)
    return -y*np.log(y_h) - (1-y)*np.log(1-y_h)

def l2(y,z):
    return -y*(np.log(sigmoid(np.abs(z))) - max(0,-z)) - (1-y)*(-z + np.log(sigmoid(np.abs(z))) - max(0,-z))

def l3(y,z):
    return -(-z + np.log(sigmoid(np.abs(z))) - max(0,-z))  - y*z

def l4(y,z):
    return  z + np.log(1 + np.exp(-np.abs(z))) + max(0,-z) -y*z

print(l1(y,z))
print(l2(y,z))
print(l3(y,z))
print(l4(y,z))

print(np.log(sigmoid(z)))
print(-np.log(1 + np.exp(-np.abs(z))) - max(0,-z))
print('----')
print(np.log(1 - sigmoid(z)))
print(-z -np.log(1 + np.exp(-np.abs(z))) - max(0,-z))

275.00671534848914
275.00671534848914
275.00671534848914
275.00671534848914
-5.006715348489118
-5.006715348489118
----
-0.006715348489118056
-0.006715348489118256


## Exercise 3: Initializing the network
Consider the fully recurrent network
$$
s(t) = W x(t) + R a(t-1) \\
a(t) = \tanh(s(t)) \\
z(t) = V a(t) \\
\hat y(t) = \sigma(z(t))
$$
for $t \in \mathbb{N}, x(t) \in \mathbb{R}^{D}, s(t) \in \mathbb{R}^{I}, a(t) \in \mathbb{R}^{I}, z(t) \in \mathbb{R}^K, \hat y(t) \in \mathbb{R}^K$ and $W, R, V$ are real matrices of appropriate sizes and $\hat a(0) = 0$. 

*Compared to the lecture notes we choose $f(x) = \tanh(x) = (e^x - e^{-x})(e^x + e^{-x})^{-1}$ and $\varphi(x) = \sigma(x) = (1+e^{-x})^{-1}$. Further, we introduced an auxiliary variable $z(t)$ and transposed the weight matrices.*

Write a function `init` that takes a `model` and integers $D, I, K$ as arguments and stores the matrices $W, R, V$ as members `model.W`, `model.R`, `model.V`, respectively. The matrices should be `numpy` arrays of appropriate sizes and filled with random values that are uniformly distributed between -0.01 and 0.01. 

In [ ]:
%matplotlib inline
import numpy as np
from scipy.special import expit as sigmoid

class Obj(object):
    pass

model = Obj()
T, D, I, K = 10, 3, 5, 1

def init(model, D, I, K):
    ########## YOUR SOLUTION HERE ##########

Obj.init = init
model.init(D, I, K)

## Exercise 4: The forward pass
Implement the forward pass for the fully recurrent network for sequence classification (many-to-one mapping). To this end, write a function `forward` that takes a `model`, a sequence of input vectors `x`, and a label `y` as arguments. The inputs will be represented as a `numpy` array of shape `(T, D)`. It should execute the behavior of the fully recurrent network and evaluate the (numerically stabilized) binary cross-entropy loss at the end of the sequence and return the resulting loss value. Store the sequence of hidden activations $(a(t))_{t=1}^T$ and the logit $z(T)$ into `model.a` and `model.z`, respectively. 

In [ ]:
def forward(model, x, y):
    ########## YOUR SOLUTION HERE ##########

Obj.forward = forward
model.forward(np.random.uniform(-1, 1, (T, D)), 1)

## Exercise 5: The computational graph

Visualize the computational graph of the fully recurrent network unfolded in time. The graph should show the functional dependencies of the nodes $x(t), a(t), z(t), L(z(t), y(t))$ for $t \in \{1, 2, 3\}$. Use the package `networkx` in combination with `matplotlib` to draw a directed graph with labelled nodes and edges. If you need help take a look at [this guide](https://networkx.guide/visualization/basics/). Make sure to arrange the nodes in a meaningful way. 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

########## YOUR SOLUTION HERE ##########